# Hora de Código: Enseñando a Aprender
## El pipeline de data science

Supongamos nos encomiendan la tarea de escribir una sequela para el Libro Don Quixote de la Mancha. Debido a nuestra poca experiencia en obras literarias, y la inifinitesimal probabilidad de que hayamos leído la obra en su totalidad.

![DonQuijote](https://www.telesurtv.net/__export/1421342197589/sites/telesur/img/multimedia/2015/01/15/quijote.jpg_1718483347.jpg)

Para lograr el objetivo, será necesario dividir la tarea en tres partes

* **Análisis**
* **Modelación**
* **Producción**

In [1]:
import re
import pickle
import requests
import numpy as np
from io import BytesIO
from unidecode import unidecode
from collections import Counter, deque

# Análisis
----
## Primeros Pasos

Con el fin de escribir una sequela, lo primero que realizaremos será entender la primera parte del libro.

**¿De qué manera podemos obtener acceso a la obra?**  
1. Transcribir el libro a nuestra computadora 
2. Buscar el libro en línea, copiarlo y pegarlo en algún lugar para tener acceso a este
3. Acceder directamente al libro y no tener que copiar y pegar nada (👍)

Para nuestra suerte, la página _[Project Gutemberg](http://www.gutenberg.org)_ ofrece libros gratuitos en línea.

Sin necesidad de acceder explicitamente a la página, podemos guardar el libro _Don Quijote_ por Miguel de Cervantes Saavedra con las siguientes líneas de código.

In [2]:
url = "http://www.gutenberg.org/cache/epub/2000/pg2000.txt"
r = requests.get(url)
# Dentro de esta variable guardamos el texto
corpus = r.text

Dado que el texto que acabamos de descargar cuenta con información adicional al libro, limpiamos los datos a fin de acotar los datos a analizar y simplificar el análisis.

In [3]:
# En las siguientes dos líneas de código buscamos el inicio y el final del libro
init_book = corpus.find("En un lugar de la Mancha")
end_book = corpus.find("End of Project Gutenberg's")
# Acotamos el libro
text = corpus[init_book: end_book]
# Removemos acentos y eliminamos salltos de línas
text = unidecode(text.replace("\r\n", " ")).lower()

In [4]:
# Observamos los primeros 500 carácteres del libro
text[:500]

'en un lugar de la mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivia un hidalgo de los de lanza en astillero, adarga antigua, rocin flaco y galgo corredor. una olla de algo mas vaca que carnero, salpicon las mas noches, duelos y quebrantos los sabados, lantejas los viernes, algun palomino de anadidura los domingos, consumian las tres partes de su hacienda. el resto della concluian sayo de velarte, calzas de velludo para las fiestas, con sus pantuflos de lo mesmo, y los dias'

El análisis que haremos a continuación consiste en analizar de una manera delimitada la manera en la que Miguel de Cervantes escribió el libro. Para esto, consideraremos cada una de las palabras dentro del texto y las guardaremos dentro de un arreglo ordenado de elementos conocido como una lista.

In [5]:
tokens = text.split()
tokens[:12]

['en',
 'un',
 'lugar',
 'de',
 'la',
 'mancha,',
 'de',
 'cuyo',
 'nombre',
 'no',
 'quiero',
 'acordarme,']

Con la información manipulada hasta el momento, sería una buena idea ver qué palabras son las que más se repiten dentro del texto.

**¿Qué palabras esperaríamos que se repitieran un mayor número de veces?**

In [6]:
# Contamos cada una de de las palabras dentro de la lista
word_counter = Counter(tokens)

**Corre la siguiente celda para observar las palabras que más se repiten**

In [ ]:
# Observamos los 10 elemento que más se repiten
word_counter.most_common(10)

Podemos acceder al número de veces que se repite una palabra en específico de la siguiente manera:
```python
word_counter["palabra"]
```
Dónde `"palabra"` es la palabra a buscar

In [7]:
word_counter["dulcinea"]

163

**¿Cuántas veces se repite la palabra `"quijote"`?**

**¿Cuántas veces se repite la palabra `"amigo"`?**

Observando el resultado de `word_counter.most_common(10)`, vemos que las palabras que más se repiten son redundantes para hacer un análisis del texto.

Dentro del archivo `"spanish_stop.pkl"` guardamos una lista con palabras redundantes en español. Corre la siguiente celda. **¿Qué observas?**

In [ ]:
spanish_stopwords = pickle.load(open("spanish_stop.pkl", "rb"))
spanish_stopwords[:10]

La siguiente celda filtrará los elementos que se encuentren dentro de la variable `spanish_stopwords`. **¿Qué palabras crees se repetirán más bajo este contexto?**

Corre la siguiente celda para averiguar las 10 palabras que más se repiten, filtrando todas aquellas palabras que se encuentren deentro de la lista `spanish_stopwords`.

In [ ]:
clean_tokens = [t for t in tokens if t not in spanish_stopwords]
clean_counter = Counter(clean_tokens)
clean_counter.most_common(10)

### Hagamos una imágen con las palabras que más se repiten

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
wordcloud = WordCloud().generate(" ".join(clean_tokens))
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

## Los $n$-grams
Saber las palabras que más se repiten no ofrece mucho contexto sobre la trama del libro. A fin de obtener un poco más de contexto sobre el libro contaremos _pares_ ordenados de palabras.

In [9]:
# El siguiente código llena la lista "elements" con pares de
# palabras dentro del libro
elements = []
for w0, w1 in zip(tokens[0:-1], tokens[1:len(tokens)]):
    element = (w0, w1)
    elements.append(element)

In [10]:
elements[:5]

[('en', 'un'),
 ('un', 'lugar'),
 ('lugar', 'de'),
 ('de', 'la'),
 ('la', 'mancha,')]

Al correr la siguiente celda, nos mostrará los pares de palabras con más repeticiones dentro del libro

In [ ]:
bigrams = Counter(elements)
bigrams.most_common(10)

Hasta ahora tenemos lo _bigram_ más comúnes, **¿de qué manera podemos conocer los _bigrams_ que empiecen con ciertas palabras?**


In [ ]:
topv = sorted(filter(lambda w: w[0] == "don", bigrams),
                     key=lambda w: bigrams[w])[:-5:-1]
for v in topv:
    print(v, bigrams[v])

¿De qué manera podríamos calcular la probabilidad de que Cervantes haya escrito `"quijote"` dado que la palabra precedente a esta es `"don"`?

$$
    \mathbb{P}(\texttt{"quijote"} | \texttt{"don"})
$$

In [88]:
topv = sorted(filter(lambda w: w[1] == "quijote", bigrams),
                     key=lambda w: bigrams[w])

wfreq = 0
count = 0
for v in topv:
    if v[0] == "don":
        wfreq = bigrams[v]
    count += bigrams[v]
print(wfreq / count)

0.9981421272642824


Dichos pares ordenados de palabras dentro de un texto se conocen como _bigrams_. En caso de tener tercias de palabras, estos se conocen como _trigrams_.

En general,  $n$ palabras ordenadas dentro un texto se conocen como $n$-grams.

### Generalización
Al igual que con las matemáticas, la generalización de un problema es de suma importancia. En el caso de la programación, una manera manera de generalizar un problema es mediante la creación de una función.

Para el análisis que hemos estado realizando, nos gustaría definir una función que nos arroje los $n$-grams del libro

In [12]:
def make_ngrams(tokens, ngram=2):
    ntokens = len(tokens)
    groups = [
        tokens[slice(i, ntokens - ngram + i )]
    for i in range(ngram)]
    grams = [ws for ws in zip(*groups)]
    return grams

In [13]:
for ws in make_ngrams(tokens, ngram=2)[:5]:
    print(ws)

('en', 'un')
('un', 'lugar')
('lugar', 'de')
('de', 'la')
('la', 'mancha,')


In [14]:
g5 = Counter(make_ngrams(tokens, ngram=5))
g5.most_common(10)

[(('don', 'quijote', 'de', 'la', 'mancha,'), 79),
 (('don', 'quijote', 'de', 'la', 'mancha'), 25),
 (('en', 'todos', 'los', 'dias', 'de'), 21),
 (('el', 'caballero', 'de', 'la', 'triste'), 21),
 (('caballero', 'don', 'quijote', 'de', 'la'), 17),
 (('de', 'don', 'quijote', 'de', 'la'), 16),
 (('senor', 'don', 'quijote', 'de', 'la'), 16),
 (('la', 'sin', 'par', 'dulcinea', 'del'), 14),
 (('todos', 'los', 'dias', 'de', 'mi'), 14),
 (('el', 'cura', 'y', 'el', 'barbero'), 13)]

In [15]:
topv = sorted(filter(lambda w: w[0] == "don" and w[1] == "fernando", g5),
                     key=lambda w: g5[w])[:-5:-1]
for v in topv:
    print(v, g5[v])

('don', 'fernando', 'y', 'sus', 'camaradas,') 2
('don', 'fernando', 'y', 'a', 'los') 2
('don', 'fernando', 'de', 'guevara,', 'donde') 1
('don', 'fernando', 'al', 'cura', 'donde') 1


# Modelación

## Modelando con $n$-grams 

Para modelar con $n$-grams, hacemos la siguiente suposición.
$$
    \mathbb{P}(w_M | w_{M-1}, w_{M-2}, \ldots, w_{1}) = \mathbb{P}(w_M | w_{M-1}, w_{M-2}, \ldots, w_{M -{N+1}})
$$

**¿Qué pasa cuando $n=2$?**


* Al modelar nuestro sistema con $n$-grams es necesario guardar y calcular, a cada iteración, la probabilidad de las palabras
* ¿Qué sucede si no existe probabilidad conocida?

In [16]:
from numpy.random import choice, seed

In [17]:
def filter_tokens(tokens, counter):
    """
    For a list tokens of size n (len(tokens) == n),
    estimate all next n+1 words following tokens. If there are
    none, return an empty list
    """
    probs = []
    elements = []
    for v in counter:
        if " ".join(tokens) == " ".join(v[:-1]):
            elements.append(v[-1])
            probs.append(counter[v])
    return elements, np.array(probs) / sum(probs)

def next_word(seed_tokens, tokens_corpus):
    """
    Estimate next word based on the "seed_tokens". If
    there is no len(seed_tokens) + 1 ngram that matches the
    seed tokens, reduce the token one element up until you
    simply estimate a random word
    """
    skip = 0
    empty = True
    n = len(seed_tokens) + 1
    while empty:
        grams = make_ngrams(tokens_corpus, ngram=n)
        gram_counter = Counter(grams)
        e, p = filter_tokens(seed_tokens[skip:], gram_counter)
        if len(e) > 0:
            empty = False
        else:
            skip += 1
            n -= 1
    word = choice(e, p=p)
    return word

def write(seed_str, corpus_tokens, nwords=15, verbose=False):
    seed_tokens = seed_str.split()
    for _ in range(nwords):
        word = next_word(seed_tokens, corpus_tokens)
        seed_tokens.append(word)
        if verbose:
            print(" ".join(seed_tokens))
    return " ".join(seed_tokens)

**Corre la siguiente celda para empezar a escribir nuestra nueva versión del libro usando _n-grams_**.

In [ ]:
seed(1643)
write("en un lugar", tokens, verbose=True)

En el ejemplo anterior, nuestro cuento empieza con `"en un lugar"`. Modifica la siguiente celda para crear un inicio alternativo.

In [ ]:
write("**aquí va tu string**", tokens, verbose=True)

#### Las desventajas de un modelo $n$-gram
* A medida que nuestro cuento crece, el tiempo de respuesta incrrementa
* Es necesario tener toda la información a todo momento para poder hacer uso de esta

## Modelando con LSTMs

Una manera alternativa de escribir nuestro libro es _entrenar_ un modelo de redes neuronales que, de alguna manera, entienda semántica y estructura de la escritura.

En terminología de machine learrning, decimos que _entrenamos_ un modelo paramétrico cuando hacemos cualquier proceso a fin de llegar a los pesos $\theta$ del modelo que minimicen cierto error $\mathcal L$.

La arquitectura de redes neuronales que necesitamos para solucionar este problema es conocida como RNN (Recurrent Neural Network). Específicamente, ocuparemos un _estilo_ del celda conocido como LSTM (Long-Short Term Memory) que nos ayude a escribir nuestro libro

![LSTM](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

Los siguientes dos links son una buena introducción hacía el poder de los RNNs
* https://colah.github.io/posts/2015-08-Understanding-LSTMs/
* http://karpathy.github.io/2015/05/21/rnn-effectiveness/

In [19]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

Entrenar un modelo con un LSTM requiere de miles de parámetros y cientos de iteraciones para converger. Sin embargo, la ventaja de entrenar un LSTM es la flexibilidad que nos da para entrenar el modelo.

Para este ejmplo, en lugar de entrenar el modelo palabra por palabra, lo haremos caractér por caractér.

* **¿Qué ventajas tendrá entrenar un modelo de esta manera?** 
* **¿Qué desventajas tendrá entrenar un modelo de esta manera?** 

Para este nueva variante del modelo, en lugar de necesitar palabras iniciales, el modelo necesitará un número específico de carácteres iniciales. En nuestro caso 60.

In [20]:
lenght = 60
sequences = [text[ix-lenght: ix+1] for ix in range(lenght, len(text))]
ch_ix = pickle.load(open("encoding_dict.pkl", "rb"))

A fin de tener una manera numérica de manipular la información, codificaremos cada caractér por un valor único otorgado por nuestro programa.

Asumiendo que el cuento tiene $S$ sequencias y $E$ carácteres por sequencia, transformaremos nuestros datos a fin de tener una matriz $S\times E$.

In [27]:
sequences[:10]

['en un lugar de la mancha, de cuyo nombre no quiero acordarme,',
 'n un lugar de la mancha, de cuyo nombre no quiero acordarme, ',
 ' un lugar de la mancha, de cuyo nombre no quiero acordarme, n',
 'un lugar de la mancha, de cuyo nombre no quiero acordarme, no',
 'n lugar de la mancha, de cuyo nombre no quiero acordarme, no ',
 ' lugar de la mancha, de cuyo nombre no quiero acordarme, no h',
 'lugar de la mancha, de cuyo nombre no quiero acordarme, no ha',
 'ugar de la mancha, de cuyo nombre no quiero acordarme, no ha ',
 'gar de la mancha, de cuyo nombre no quiero acordarme, no ha m',
 'ar de la mancha, de cuyo nombre no quiero acordarme, no ha mu']

In [28]:
len(sequences)

2071389

In [21]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
sequences_int = [[ch_ix[ch] for ch in seq] for seq in sequences]
sequences_int = np.array(sequences_int)
sequences_int

array([[27, 35,  0, ..., 34, 27,  6],
       [35,  0, 42, ..., 27,  6,  0],
       [ 0, 42, 35, ...,  6,  0, 35],
       ...,
       [46,  0, 30, ...,  0,  0,  0],
       [ 0, 30, 23, ...,  0,  0,  0],
       [30, 23, 35, ...,  0,  0,  0]])

In [22]:
sequences_int.shape

(2071389, 61)

### Aprendiendo a Escribir

Finalmente, para enseñarle a nuestro modelo a escribir, modelamos nuestro problema de la siguiente manera: dado $n-1$ carácteres, queremos que el modelo estime el $n$-ésimo carácter.

Para el caso de una red neuronal, así como en varios problemas en machine learning, la codificación categorica de un valor a un entero puede ocasionar resultados subóptimos al momento de entrenar el modelo. Para evitar esto, transformaremos cada carácter a un vector de dimensión $C$ (siendo $C$ el número de carácteres únicos), en el cuál todos los elementos de este nuevo vector son 0, excepto por la posición

Por ejemplo, si un caractér tiene valor $2$, creamos un vector $[0, 0, 1, \ldots, 0]$; de igual manera si un carácter tiene valor $0$, creamos un vector de la forma $[1, 0, 0, \ldots, 0]$. Y así sucesivamente

In [23]:
X_train, y_train = sequences_int[:50,:-1], sequences_int[:50, -1:]

X_train = to_categorical(X_train, num_classes=vocab_size)
y_train = to_categorical(y_train, num_classes=vocab_size)

Bajo estas dos consideraciones, obtenemos una matriz $S\times C$ de carácteres por estimar y un arreglo 3-dimensional $S\times E \times C$ de datos dependientes.

In [34]:
y_train.shape

(50, 48)

In [29]:
X_train.shape

(50, 60, 48)

#### Definimos una pequeña arquitectura de modelo

In [24]:
X_input = Input(X_train.shape[1:])
X = LSTM(100, activation="relu", return_sequences=False)(X_input)
X = Dense(vocab_size, activation="softmax")(X)
model = Model(inputs=X_input, outputs=X)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 60, 48)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               59600     
_________________________________________________________________
dense (Dense)                (None, 48)                4848      
Total params: 64,448
Trainable params: 64,448
Non-trainable params: 0
_________________________________________________________________


#### Entrenamos el modelo

In [25]:
optimizer = Adam(lr=0.0001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
50/50 [==============================] - 2s 43ms/step - loss: 3.8622 - acc: 0.0200
Epoch 2/5
50/50 [==============================] - 0s 5ms/step - loss: 3.8604 - acc: 0.0200
Epoch 3/5
50/50 [==============================] - 0s 5ms/step - loss: 3.8587 - acc: 0.0200
Epoch 4/5
50/50 [==============================] - 0s 5ms/step - loss: 3.8568 - acc: 0.0200
Epoch 5/5
50/50 [==============================] - 0s 5ms/step - loss: 3.8551 - acc: 0.0200


Una vez entrenado el modelo, podemos hacer inferencia sobre nuestro nuevo libro

In [35]:
decoding = {val:char for char, val in ch_ix.items()}
n_chars = len(ch_ix)
seq = "despues de haber vivido su primera aventura don quijote se sentia"
text_seq = deque([ch for ch in seq], maxlen=lenght)

for _ in range(100):
    encoded = [ch_ix[ch] for ch in text_seq]
    encoded = pad_sequences([encoded], maxlen=lenght, padding="pre")
    encoded = to_categorical(encoded, num_classes=n_chars).reshape(1, -1, n_chars)
    probs = model.predict(encoded, batch_size=1)
    # pred = np.argmax(model.predict(encoded, batch_size=1))
    pred = np.random.choice(np.arange(n_chars), p=probs.ravel())
    char = decoding[pred]
    text_seq.append(char)
    seq += char

In [36]:
print(seq)

despues de haber vivido su primera aventura don quijote se sentia??yv2u'.)roes10v]h(0tn3aho-?uz ;>"7du-az4-yinj6xs(6 :n?ma,;]gg;'s'4)w0em" ?hu'stsg1i'goy!e"(>wsb,uz4


Al parecer nuestro modelo no es tan bueno para escribir un libro...

Esto último se debe a que necesitamos varias épocas de entrenamiento para llegar un mejor modelo que pueda escribir un libro. Los siguientes ejemplos muestran la vida del modelo después de varias épocas

* ~ 5 epochs (5 secuencias):  
`despues de haber vivido su primera aventura don quijote se sentia??yv2u'.)roes10v]h(0tn3aho-?uz ;>"7du-az4-yinj6xs(6 :n?ma,;]gg;'s'4)w0em" ?hu'stsg1i'goy!e"(>wsb,uz4`

* ~ 50 epochs:   
  `despues de haber vivido su primera aventura don quijote se sentias eldeno- uelvimes o`
  
* ~ 150 epochs:   
  `despues de haber vivido su primera aventura don quijote se sentia serentas tan ojo es busa coma estado este amanas. ahardada digustra mando, con somparia de egla fue``
  
* ~ 190 epochs:   
  `despues de haber vivido su primera aventura don quijote se sentia de aris, dilino; ya a oflico el estria don quiso y nuesa.  y si no tenda trezas que tienele y de lg`